In [7]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML
from tmdbv3api import TMDb, Genre
from openai import AsyncOpenAI
from tmdbv3api import TMDb
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function
from tmdbv3api import TMDb, Discover
from tmdbv3api import Movie
import pandas as pd
from datetime import datetime
from tmdbv3api import TMDb, Discover
import os, json
from tmdbv3api import TMDb, Genre, Discover
import semantic_kernel as sk

load_dotenv()

True

In [9]:
tmdb = TMDb()
tmdb.api_key=os.environ.get('TMDB_KEY')

In [11]:
# Initialize TMDb with your API key
tmdb = TMDb()
tmdb.api_key ='2fa30f6a1d22eb80c6dc9cac9cc67bdc'
# Fetch the official list of movie genres
genre_client = Genre()
all_genres = genre_client.movie_list()

# Build a name→ID lookup (lowercased keys)
GENRE_MAP = {g.name.lower(): g.id for g in all_genres}

# Create a Discover instance for querying movies
discover = Discover()

client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [16]:
def recommend_movies(
    genre_id: int,
    preference: str = "newer",
    top_n: int    = 5,
    weight_pop: float = 0.5,
    weight_vote: float = 0.5
):
    """
    Fetch page 1 for `genre_id`, score by weighted(popularity, vote_average),
    filter by age (≤5 yrs new vs >5 yrs old), return top_n dicts.
    """
    current_year = datetime.now().year
    raw = discover.discover_movies({
        "with_genres": genre_id,
        "sort_by":     "popularity.desc",
        "page":        1
    })
    scored = []
    for m in raw:
        try:
            year = int(m.release_date[:4])
        except:
            continue
        age = current_year - year
        if preference == "newer" and age > 5:  continue
        if preference == "older" and age <= 5: continue
        score = weight_pop * m.popularity + weight_vote * m.vote_average
        scored.append({"title": m.title, "year": year, "score": score})
    return sorted(scored, key=lambda x: x["score"], reverse=True)[:top_n]

In [17]:
class MoviePlugin:
    @kernel_function(description="Recommend movies by genre (name or ID), date preference, and number of results")
    def recommend(
        self,
        genre: str,
        preference: str = "newer",
        top_n: str    = "5"
    ) -> str:
        if genre.isdigit():
            gid = int(genre)
        else:
            gid = GENRE_MAP.get(genre.lower())
            if gid is None:
                return (
                    f"Unknown genre '{genre}'. "
                    f"Available: {', '.join(GENRE_MAP.keys())}"
                )
        try:
            n = int(top_n)
        except:
            n = 5

        recs = recommend_movies(genre_id=gid, preference=preference, top_n=n)

        return "\n".join(
            f"{i+1}. {m['title']} ({m['year']}) — score={m['score']:.2f}"
            for i, m in enumerate(recs)
        )

In [18]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[MoviePlugin()],
    name="MovieAgent",
    instructions="You will identify the preferred genres that the customer wants based on their prompt",
)


In [19]:
user_inputs = [
    "I like comedy movies recommend me some.",
    "I don't like that one, please show me horror.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()